# 50.039 Theory and Practice of Deep Learning Project 2024

Group 10
- Issac Jose Ignatius (1004999)
- Mahima Sharma (1006106)
- Dian Maisara (1006377)


## Motivation

Chest radiography is an essential diagnostic tool used in medical imaging to visualise structures and organs within the chest cavity. It is crucial for diagnosing various respiratory and heart-related conditions. However, with the increased demand for radiological reports within shorter timeframes to detect and treat illnesses, there have been insufficient radiologists available to perform such tasks at scale. Therefore, automated chest radiograph interpretation could provide substantial benefits supporting large-scale screening and population health initiatives. Deep-learning algorithms can be used to bridge this gap. They have been used for image classification, anomaly detection, organ segmentation, and disease progression prediction.
<br><br>

*In this project, we aim to train a deep neural network to perform multi-label image classification on a wide array of chest radiograph images that exhibit various pathologies.*<br><br>



---




## Import all relevant libraries

In [1]:
# Numpy
import numpy as np
# Pandas
import pandas as pd

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision
import torchvision.transforms as T
from torchvision.transforms import v2
from torchvision.io import read_image, ImageReadMode
from torchmetrics.classification import BinaryAccuracy


# File Operations
import os

# Helper scripts
from tqdm.notebook import tqdm, tnrange
import math
# import sys
# sys.path.insert(0, '../src')
# from saver_loader import *
# %reload_ext autoreload
# %autoreload 2

print(torchvision.__version__)

0.17.2+cu121


In [2]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu") 
print(device)

cuda


## Data Loading

The training and validation datasets are from the **CheXphoto dataset** (Philips et al., 2020). <br><br> CheXphoto comprises a training set of natural photos and synthetic transformations of 10,507 X-ray images from 3,000 unique patients (32,521 data points) sampled at random from the CheXpert training dataset and an accompanying validation set of natural and synthetic transformations applied to all 234 X-ray images from 200 patients with an additional 200 cell phone photos of x-ray films from another 200 unique patients (952 data points).

### Retrieving dataset from Google Cloud Storage (GCS)




In [3]:
# # Connect to GCS to access data
# from google.colab import auth
# auth.authenticate_user() # TODO: everyone to send me gmail so I can have you authed for bucket access

# project_id = 'tpdl-414711'
# bucket_name = 'chexphoto-v1'
# !gcloud config set project {project_id}

# # Install Cloud Storage FUSE.
# !echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
# !apt -qq update && apt -qq install gcsfuse

# # Mount a Cloud Storage bucket or location, without the gs:// prefix.
# mount_path = "chexphoto-v1"  # or a location like "my-bucket/path/to/mount"
# local_path = f"/mnt/gs/{mount_path}"

# !mkdir -p {local_path}
# !gcsfuse --implicit-dirs {mount_path} {local_path}

### Setup environment variables


In [4]:
data_path = os.path.join(os.path.abspath(''), "../ChexPhoto/chexphoto-v1")
print(data_path)

c:\Users\User\Desktop\50.039 TPDL\2024_TPDL\notebooks\../ChexPhoto/chexphoto-v1


### Loading dataset (image and labels)

In [5]:
# Bug in Path present in training dataset
def fix_error_paths(row):
    row = row.replace("//", "/")
    return row

def str_to_array(row):
    ndarray = np.fromstring(
                row.replace('\n','')
                    .replace('[','')
                    .replace(']','')
                    .replace('  ',' '), 
                    sep=' ')
    return ndarray

In [6]:
train_df = pd.read_csv("../data/processed/train_one_hot_encoded.csv", index_col=False)
labels = train_df.columns[1:]
print(len(labels), labels)

13 Index(['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')


In [7]:
train_df["Path"] = train_df["Path"].apply(fix_error_paths)

valid_df = pd.read_csv("../data/processed/valid_one_hot_encoded.csv", index_col=False)
#test_df = pd.read_csv("../data/processed/test_one_hot_encoded.csv", index_col=False)

for label in labels:
    train_df[label] = train_df[label].apply(str_to_array)
    valid_df[label] = valid_df[label].apply(str_to_array)
    #test_df[label] = test_df[label].apply(str_to_array)

display(train_df)
display(valid_df)
#display(test_df)

,Path,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
1,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
2,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
3,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
4,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32516,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
32517,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
32518,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
32519,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"


,Path,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
1,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
2,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
3,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
4,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
698,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
699,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
700,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"


### Subsetting dataset (Binary Classification for Pleural Effusion)


In [8]:
def keep_observations(df, cols):
    return df[cols].copy()

In [9]:
# Drop all other labels
cols = ["Path", "Pleural Effusion", "Cardiomegaly"]
pEff_train = keep_observations(train_df, cols)
pEff_valid = keep_observations(valid_df, cols)
# pEff_test = keep_observations(test_df, cols)

In [10]:
def ohe_to_class(row):
    if np.sum(row) > 0:
        return np.argmax(row)
    else:
        return -100

pEff_train["Class_pEff"]= pEff_train["Pleural Effusion"].apply(ohe_to_class)
pEff_train["Class_Cardio"]= pEff_train["Cardiomegaly"].apply(ohe_to_class)

pEff_valid["Class_pEff"]= pEff_valid["Pleural Effusion"].apply(ohe_to_class)
pEff_valid["Class_Cardio"]= pEff_valid["Cardiomegaly"].apply(ohe_to_class)

In [11]:
display(pEff_train)
display(pEff_valid)
# display(pEff_test)

,Path,Pleural Effusion,Cardiomegaly,Class_pEff,Class_Cardio
0,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100
1,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 0.0]",1,-100
2,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",-100,-100
3,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]",-100,-100
4,CheXphoto-v1.0/train/synthetic/digital/patient...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]",2,2
...,...,...,...,...,...
32516,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32517,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32518,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100
32519,CheXphoto-v1.0/train/natural/nokia/patient6446...,"[0.0, 0.0, 1.0]","[0.0, 0.0, 0.0]",2,-100


,Path,Pleural Effusion,Cardiomegaly,Class_pEff,Class_Cardio
0,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]",1,2
1,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
2,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
3,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
4,CheXphoto-v1.0/valid/synthetic/digital/patient...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
...,...,...,...,...,...
697,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
698,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1
699,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]",1,2
700,CheXphoto-v1.0/valid/natural/oneplus/patient64...,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]",1,1


In [12]:
# def sum_array(row):
#     return np.sum(row)

# pEff_train["Pleural Effusion_sum"] = pEff_train["Pleural Effusion"].apply(sum_array)
# pEff_train["Cardiomegaly_sum"] = pEff_train["Cardiomegaly"].apply(sum_array)

# display(pEff_train)

In [13]:
# pEff_train = pEff_train[(pEff_train["Pleural Effusion_sum"] > 0) | (pEff_train["Cardiomegaly_sum"] > 0)]

# # filter out NaNs given that we feed class labels to CrossEntropyLoss
# pEff_train = pEff_train[cols]
# display(pEff_train)

### Custom Dataset implementation

In [14]:
# Implementation of Custom Dataset Class for CheXPhoto Dataset
class CheXDataset(Dataset):
    # Accepts dataframe object and str
    def __init__(self, df: pd.DataFrame, px_size: int = 256):
        self.dataframe = df.copy()
        self.px_size = px_size

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x_path = data_path + "/" + self.dataframe.iloc[idx, 0].split("CheXphoto-v1.0", 1)[-1]
        
        transform = T.Compose([
            v2.Resize((self.px_size, self.px_size), interpolation=T.InterpolationMode.BICUBIC)
        ])

        resized_x_tensor = transform(read_image(x_path, mode = ImageReadMode.RGB)) /255

        # FIXME: Changed from 1: to 3:
        # y = torch.tensor(self.dataframe.iloc[idx, 1:], dtype=torch.float32)
        y = torch.tensor(self.dataframe.iloc[idx, 3:]).type(torch.LongTensor)
        return resized_x_tensor, y

### Custom Dataloader

In [15]:
# Load into custom Dataset
pEff_train_data = CheXDataset(pEff_train)
pEff_valid_data = CheXDataset(pEff_valid)
#pEff_test_data = CheXDataset(pEff_test)

# Prepare random sampler for training subset [19582/1873, 19582/4976, 19582/12733]
train_sampler = WeightedRandomSampler([1873/19582, 4976/19582, 12733/19582], int(len(pEff_train_data)))

# Load into DataLoader
batch_size = 128
pEff_train_loader = DataLoader(pEff_train_data, batch_size, sampler=train_sampler)
pEff_valid_loader = DataLoader(pEff_valid_data, batch_size)
#pEff_test_loader = DataLoader(pEff_test_data, batch_size)

In [16]:
x, y = pEff_train_data[0]
print(x, x.shape, x.dtype)
print(y, y.shape, y.dtype)

tensor([[[0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         ...,
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588]],

        [[0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         ...,
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588]],

        [[0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.0588],
         [0.0588, 0.0588, 0.0588,  ..., 0.0588, 0.0588, 0.

C:\Users\User\AppData\Local\Temp\ipykernel_27948\2802765927.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = torch.tensor(self.dataframe.iloc[idx, 3:]).type(torch.LongTensor)


## Model Tuning

Our initial model is a simple feedforward neural network with multiple heads (2 heads) capable of classifying for both Cardiomegaly and Pleural Effusion. We will utilise the Cross-entropy loss function to optimise the model during training.

**This is a TODO since it can change**


### First iteration - Simple Convolutional Neural Network

#### Model

In [17]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int, pool_size: int = 2):
        super(ConvolutionBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()
        self.pooling = nn.MaxPool2d(pool_size, pool_size)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.activation(out)
        out = self.pooling(out)
        return out

In [18]:
class FCBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, dropout_rate: int=0.2):
        super(FCBlock, self).__init__()
        
        self.fc = nn.Linear(in_features, out_features, dtype=torch.float32)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        out = self.fc(x)
        out = self.activation(out)
        out = self.dropout(out)
        return out

In [19]:
class ClassifierBlock(nn.Module):
    def __init__(self, in_features: int, hidden_features: tuple[int], out_features: int, dropout_rate: int = 0.2):
        super(ClassifierBlock, self).__init__()

        self.inputs = FCBlock(in_features, hidden_features[0], dropout_rate)
        self.layers = nn.Sequential(*[FCBlock(hidden_features[i], hidden_features[i+1], dropout_rate) for i in range(len(hidden_features)-1)])
        self.linear = nn.Linear(hidden_features[-1], out_features, dtype=torch.float32)
    
    def forward(self, x):
        out = self.inputs(x)
        out = self.layers(out)
        out = self.linear(out)
        return out

In [20]:
class ConvolutionalNN(nn.Module):
    def __init__(self, channels: tuple[int], img_size: tuple[int], hidden_size: tuple[int], output_size: int, dropout_rate: int = 0.2):
        super(ConvolutionalNN, self).__init__()
        
        # Hyperparameters for Convolutional Layers
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.pool_size = 2

        # Input size of Image
        self.input_height, self.input_width = img_size

        # Calculate Input Size of Classifier
        for i in range(len(channels) - 1):
            self.input_height = math.floor((self.input_height + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            self.input_width =  math.floor((self.input_width  + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
        
        self.classifier_size = channels[-1] * self.input_height * self.input_width
        
        # Model Layers
        self.conv_layers = nn.Sequential(*[ConvolutionBlock(channels[i], channels[i+1], self.kernel_size, self.stride, self.padding, self.pool_size) for i in range(len(channels)-1)])
        self.classifier1 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)


    def forward(self,x):
        # Convolutional Layers
        out1 = self.conv_layers(x)

        # keep batch size and flatten the rest
        out2 = out1.view(out1.size(0), -1)

        # Classifier Layers
        out3 = self.classifier1(out2)
        return out3

In [21]:
# Create Model
channels = (3, 8, 16, 32, 128)
input_size = (256, 256)
hidden_size = (8192, 512)
output_size = 3
dropout_rate = 0.2

model = ConvolutionalNN(channels, input_size, hidden_size, output_size, dropout_rate).to(device)
print(model)

ConvolutionalNN(
  (conv_layers): Sequential(
    (0): ConvolutionBlock(
      (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvolutionBlock(
      (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvolutionBlock(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode

#### Training

In [22]:
def train_loop(model, train_loader, optimizer, loss):
    model.train()

    train_loss = 0.0
    train_total = 0
    train_correct = 0

    for inputs, outputs in tqdm(train_loader):
        # outputs_pEff = outputs[:, 0, :]
        outputs_pEff = outputs[:, 0]
        inputs_re, outputs_re = inputs.to(device), outputs_pEff.to(device)
        
        optimizer.zero_grad()
        preds = model(inputs_re)

        # Do not feed class labels as NaNs will be computed as class 0 during training. Instead, need to feed one-hot encoded labels.
        # loss_value = loss(preds, torch.argmax(outputs_re, dim=1))
        loss_value = loss(preds, outputs_re)
        loss_value.backward()
        optimizer.step()

        # Compute metric
        train_loss += loss_value.item() * outputs_re.size(0)
        train_total += outputs_re.size(0)
        # train_correct += (torch.argmax(preds, dim=1) == torch.argmax(outputs_re, dim=1)).sum().item() # Convert both to class labels
        train_correct += (torch.argmax(preds, dim=1) == outputs_re).sum().item()

    train_loss /= train_total
    train_accuracy = train_correct / train_total

    return train_loss, train_accuracy

### TODO: Implementation Validation Loss

In [23]:
def test_loop(model, valid_loader):
    model.eval()
        
    val_total = 0
    val_correct = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, outputs in tqdm(valid_loader):
            # Retrieve predictions
            # outputs_pEff = outputs[:, 0, :]
            outputs_pEff = outputs[:, 0]
            inputs_re, outputs_re = inputs.to(device), outputs_pEff.to(device)
            preds = model(inputs_re)

            # Compute metrics
            val_total += outputs_re.size(0)
            # val_correct += (torch.argmax(preds, dim=1) == torch.argmax(outputs_re, dim=1)).sum().item() # Convert both to class labels
            val_correct += (torch.argmax(preds, dim=1) == outputs_re).sum().item() # Convert both to class labels
            val_loss +=


    ## TODO: Implement Validation Loss as well
    
    val_accuracy = val_correct / val_total

    return val_accuracy   

In [24]:
def train(model, train_loader, valid_loader, epochs=10, lr=1e-3):
    # Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # loss function
    loss = nn.CrossEntropyLoss().to(device)

    #train_loss_values = []
    #train_accuracy_values = []
    for epoch in tnrange(epochs):
        # Train loop
        train_loss, train_accuracy = train_loop(model, train_loader, optimizer, loss)

        # Test loop
        val_accuracy = test_loop(model, valid_loader)

        ## TODO: Implement Validation Loss as well

        print(f'--- Epoch {epoch+1}/{epochs}: Train loss: {train_loss:.4f}, Train accuracy: {train_accuracy:.4f}\n Validation accuracy: {val_accuracy}')


In [25]:
train(model, pEff_train_loader, pEff_valid_loader, epochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/255 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp\ipykernel_27948\2802765927.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = torch.tensor(self.dataframe.iloc[idx, 3:]).type(torch.LongTensor)


  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 1/5: Train loss: 0.0041, Train accuracy: 0.3470
 Validation accuracy: 0.7136752136752137


...

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 4/5: Train loss: 0.0000, Train accuracy: 0.3535
 Validation accuracy: 0.7136752136752137


  0%|          | 0/255 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 5/5: Train loss: 0.0000, Train accuracy: 0.3564
 Validation accuracy: 0.7136752136752137


### Second iteration - Basic Convolutional neural network (CNN)

#### Model

In [26]:
# We will be using Convolutional Neural Network
class ConvolutionalNNv2(nn.Module):
    def __init__(self, channels: tuple[int], img_size: tuple[int], hidden_size: tuple[int], output_size: int, dropout_rate: int = 0.2):
        super(ConvolutionalNNv2, self).__init__()
        
        # Hyperparameters for Convolutional Layers
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.pool_size = 2

        # Input size of Image
        self.input_height, self.input_width = img_size

        # Calculate Input Size of Classifier
        for i in range(len(channels) - 1):
            self.input_height = math.floor((self.input_height + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            self.input_width =  math.floor((self.input_width  + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            
        self.classifier_size = channels[-1] * self.input_height * self.input_width


        # Conv Layers + Skip Connections
        self.conv1 = ConvolutionBlock(channels[0], channels[1], self.kernel_size, self.stride, self.padding, self.pool_size)
        self.conv2 = ConvolutionBlock(channels[1], channels[2], self.kernel_size, self.stride, self.padding, self.pool_size)

        # TODO: Try out 2 different skip connection implementation
        # self.skip_connection1 = nn.Sequential(nn.Conv2d(channels[0], channels[2], kernel_size=1, stride=2*self.stride, padding=self.padding),
        #                                       nn.AvgPool2d(self.pool_size, self.pool_size))
        
        self.skip_connection1 = nn.Sequential(nn.Conv2d(channels[0], channels[2], kernel_size=1, stride=self.stride, padding=self.padding),
                                              nn.AvgPool2d(2*self.pool_size, 2*self.pool_size)
                                              )
        

        self.conv3 = ConvolutionBlock(channels[2], channels[3], self.kernel_size, self.stride, self.padding, self.pool_size)
        self.conv4 = ConvolutionBlock(channels[3], channels[4], self.kernel_size, self.stride, self.padding, self.pool_size)
        
        # self.skip_connection2 = nn.Sequential(nn.Conv2d(channels[2], channels[4], kernel_size=1, stride=2*self.stride, padding=self.padding),
        #                                       nn.AvgPool2d(self.pool_size, self.pool_size))
        
        self.skip_connection2 = nn.Sequential(nn.Conv2d(channels[2], channels[4], kernel_size=1, stride=self.stride, padding=self.padding),
                                              nn.AvgPool2d(2*self.pool_size, 2*self.pool_size)
                                              )

        # Dropout Layer
        self.dropout = nn.Dropout(dropout_rate)

        # Classifier Layers
        self.classifier1 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)
        

    def forward(self,x):
        # First convolutional layer
        x1 = self.conv1(x)

        # Second convolutional layer
        x2 = self.conv2(x1)

        # Apply skip connection after the second convolutional layer
        skip_connection_output1 = self.skip_connection1(x)
        # print(self.skip_connection1(x).shape, x2.shape)
        x2 += skip_connection_output1

        # Third convolutional layer
        x3 = self.conv3(x2)

        # Fourth convolutional layer -> pooling
        x4 = self.conv4(x3)

        # Apply skip connection after the fourth convolutional layer
        skip_connection_output2 = self.skip_connection2(x2)
        # print(self.skip_connection2(x2).shape, x4.shape)
        x4 += skip_connection_output2

        # Flatten output of convolutions
        x5 = x4.view(x4.size(0), -1)
        # print(x.shape)
        x6 = self.dropout(x5)

        # Classifier layer
        x7 = self.classifier1(x6)
        # x8 = self.softmax(x7) # No need, Refer to https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss

        return x7

In [27]:
# Create Model
# model = torchvision.models.resnet18(weights=False).to(device) # to check if trainer works

channels = (3, 8, 16, 32, 128)
img_size = (256, 256)
hidden_size = (8192, 512)
output_size = 3
dropout_rate = 0.2

model = ConvolutionalNNv2(channels, img_size, hidden_size, output_size, dropout_rate).to(device)
print(model)

ConvolutionalNNv2(
  (conv1): ConvolutionBlock(
    (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): ConvolutionBlock(
    (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
    (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (skip_connection1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (1): AvgPool2d(kernel_size=4, stride=4, padding=0)
  )
  (conv3): ConvolutionBlock(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [28]:
for inputs, outputs in pEff_train_loader:
    print(inputs.shape)
    print(outputs.shape)
    break

C:\Users\User\AppData\Local\Temp\ipykernel_27948\2802765927.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = torch.tensor(self.dataframe.iloc[idx, 3:]).type(torch.LongTensor)


torch.Size([128, 3, 256, 256])
torch.Size([128, 2])


In [29]:
train(model, pEff_train_loader, pEff_valid_loader, epochs=5)
#torch.save(model.state_dict(), 'weights.pt')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/255 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp\ipykernel_27948\2802765927.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = torch.tensor(self.dataframe.iloc[idx, 3:]).type(torch.LongTensor)


  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 1/5: Train loss: 0.0041, Train accuracy: 0.3454
 Validation accuracy: 0.7136752136752137


...

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 4/5: Train loss: 0.0000, Train accuracy: 0.3515
 Validation accuracy: 0.7136752136752137


  0%|          | 0/255 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

--- Epoch 5/5: Train loss: 0.0000, Train accuracy: 0.3497
 Validation accuracy: 0.7136752136752137


In [30]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.activation(out)
        return out

In [31]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        out1 = self.conv1(x)
        out2 = self.conv2(x)
        out3 = self.conv3(x)
        out4 = self.conv4(x)
        out = torch.cat([out1, out2, out3, out4], 1)
        return out

### Testing

## Observations

**TODO** Discuss whether its right for us to pluck all our evaluation and training together and discuss it here or break up the code without any descriptions
